In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random as rand
import numpy
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
 


0it [00:00, ?it/s]

100%|█████████▉| 169697280/170498071 [00:12<00:00, 17204719.50it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
num_epochs

20

In [0]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

In [0]:
model = CNN()
'''
STEP 4: INSTANTIATE MODEL CLASS
'''

#######################
#  USE GPU FOR MODEL  #
#######################

if torch.cuda.is_available():
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [0]:
for epoch in range(num_epochs):
    loss_values = []
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        
        inputs = inputs.cuda()
        labels = labels.cuda()

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        if epoch > 16:
            for group in optimizer.param_groups:
                for p in group['params']:
                    state = optimizer.state[p]
                    if state['step'] >= 1024:
                        state['step'] = 1000
        optimizer.step()


        # print statistics
        running_loss += loss.item()

    # Normalizing the loss by the total number of train batches
    running_loss /= len(train_loader)
    loss_values.append(running_loss)
    

    # Calculate training/test set accuracy of the existing model
    #train_accuracy = calculate_accuracy(trainloader, opt.is_gpu)
    #test_accuracy = calculate_accuracy(testloader, opt.is_gpu)

    print("Iteration: {0} | Loss: {1} ".format(epoch+1, running_loss))

    # save model
    """if epoch % 50 == 0:
        print('==> Saving model ...')
        state = {
            'net': net.module if opt.is_gpu else net,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, '../checkpoint/ckpt.t7')
        """

print('==> Finished Training ...')

Iteration: 1 | Loss: 0.05808339566970244 
Iteration: 2 | Loss: 0.05824617545632645 
Iteration: 3 | Loss: 0.05216355744097382 
Iteration: 4 | Loss: 0.05119319553580135 
Iteration: 5 | Loss: 0.04821598399989307 
Iteration: 6 | Loss: 0.04887939654150978 
Iteration: 7 | Loss: 0.04564704450918362 
Iteration: 8 | Loss: 0.041646327270660546 
Iteration: 9 | Loss: 0.042067258310038595 
Iteration: 10 | Loss: 0.04377115397481248 
Iteration: 11 | Loss: 0.04091923156287521 
Iteration: 12 | Loss: 0.0412195207105251 
Iteration: 13 | Loss: 0.03663436589867342 
Iteration: 14 | Loss: 0.03816774710640311 
Iteration: 15 | Loss: 0.03654655154934153 
Iteration: 16 | Loss: 0.036453073685057465 
Iteration: 17 | Loss: 0.03541739800781943 
Iteration: 18 | Loss: 0.025432630659313872 
Iteration: 19 | Loss: 0.02167408222029917 
Iteration: 20 | Loss: 0.023597587706986816 
==> Finished Training ...


In [0]:
correct = 0
total = 0
# Iterate through test dataset
for images, labels in test_loader:
    
    if torch.cuda.is_available():
        images = Variable(images.cuda())
    else:
        images = Variable(images)
                
    # Forward pass only to get logits/output
    outputs = model(images)
                
    # Get predictions from the maximum value
    _, predicted = torch.max(outputs.data, 1)
                
    # Total number of labels
    total += labels.size(0)
                
              
    # Total correct predictions
    if torch.cuda.is_available():
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
        correct += (predicted == labels).sum()
            
accuracy = 100 * correct / total
print("Accuracy on total test data: {}%".format(accuracy.item()))

Accuracy on total test data: 80%
